In [9]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely as shp

from pandas.api.types import is_numeric_dtype
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.offsetbox import AnchoredText
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [10]:
# Configurações gerais
measure_default = 'inch' # Medida padrão para plotagens
folder_to_save = 'smaps' # Pasta padrão para salvar os mapas do atlas
cmap_default = 'jet' # Paleta de cores padrão - Ver: https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
scheme_default = 'natural_breaks' # Método de classificação padrão - Ver: mapclassify.classifiers
k_default = 8 # Número de classes padrão


def plot_syntax_map(smap: dict, 
                    column: str = measure_default, 
                    cmap: str = cmap_default, 
                    bg_black = False, 
                    save = False, 
                    show = True):
    """
    Plota mapa sintático
    
    Parâmetros
    ----------
    smap : dict
        Retorno de make_smap()
    column : str
        Nome da coluna para classificação
    cmap : str
        Paleta de cores
        Ver: https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    bg_black : bool
        Se verdadeiro, fundo do mapa será preto
    save : bool
        Se verdadeiro, o mapa será salvo em arquivo
    show : bool
        Se verdadeiro, o mapa será exibido
    Retorno
    -------
    None
    """
    
    gdf = smap['data']
    fig, ax = plt.subplots(1, 1, figsize=(15,15))
    
    # cor do norte e da escala
    artist_color = 'k'
    
    # Verifica se o fundo deve ser preto
    if bg_black:
        ax.set_facecolor('k')
        ax.grid(False)
        # se o fundo for preto, norte e escala serão brancos
        artist_color = 'w'
    
    # Ajusta barra de legenda
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.2)
    
    # Insere título com base no dicionário fornecido
    ax.set_title('Mapa sintático de {}\nModelagem: {} | Base de dados: {}\nMedidas sintáticas: {}'
        .format(smap['place'], smap['modeling'], smap['modeling_source'], smap['syntactic_measures']))
    
    # Insere título interno
    _insert_internal_title(column, ax)
    
    # Insere escala
    _insert_scale(artist_color, ax)
    
    # Insere Norte
    _insert_north(artist_color, ax)
    
    # Plota ou salva
    plot = gdf.plot(column=column, ax=ax, legend=True, cax=cax, cmap=cmap)
    if not show:
        plt.close()
    if save:
        filepath = os.path.join(folder_to_save, "map-{}.png".format(column))
        plot.get_figure().savefig(filepath, dpi=300)

def plot_higher_values(smap: dict, 
                       column: str = measure_default, 
                       quantile: float = .9, 
                       save_as_field: bool = False):
    """
    Plota mapa destacando os x% segmentos com maiores valores de uma dada medida
    
    Parâmetros
    ----------
    smap : dict
        Retorno de make_smap() : obrigatório
    column : str
        Nome da coluna para classificação
    quantile : float
        Quantil a partir do qual os valores maiores serão destacados
    save_as_field : bool
        Se verdadeiro, será criada uma coluna no GeoDataFrame com a informação
    
    Retorno
    -------
    None
    """
    
    gdf = smap['data']
    
    quantile = round(quantile, 2)
    
    # Verifica os registros com valores acima do quantil definido para uma dada medida
    quantil = gdf[column].quantile(quantile)
    mask = gdf[column] > quantil
    
    # Verifica se deve inserir novo campo no dataframe
    if save_as_field:
        new_column = '{}_higher'.format(column)
        gdf[new_column] = False
        gdf.loc[mask, new_column] = True
        
    fig, ax = plt.subplots(1, 1, figsize=(15,15))
    
    # Insere título com base no dicionário fornecido
    as_percent = "{}% segmentos com maior valor".format(round((1 - quantile) * 100))
    ax.set_title('Mapa sintático de {}: {}\nModelagem: {} | Base de dados: {}\nMedidas sintáticas: {}'
        .format(smap['place'], as_percent, smap['modeling'], smap['modeling_source'], smap['syntactic_measures']))
    
    # Insere título interno
    _insert_internal_title(column, ax)
    
    # Cor do norte e da escala
    artist_color = 'k'
    
    # Insere escala
    _insert_scale(artist_color, ax)
    
    # Insere Norte
    _insert_north(artist_color, ax)

    # Plota
    gdf.plot(ax=ax, alpha=0.1, edgecolor='k', zorder=1)
    gdf[mask].plot(ax=ax, alpha=0.9, edgecolor='k', zorder=2)

def _load_geometry_column(df, crs, geom_column='geom'):
    """
    Converte um DataFrame com campo que contém as geometrias no formato WKT em um GeoDataFrame
    
    Parâmetros
    ----------
    df : pandas.DataFrame
        Retorno de make_smap() : obrigatório
    crs : dict
        Dicionário com sistema de coordenadas no padrão {'init': 'epsg:<int>'}
    geom_column : str
        Nome da coluna que contém as geometrias
    
    Retorno
    -------
    geopandas.GeoDataFrame
    """
    
    try:
        # Garante que são strings
        df[geom_column] = df[geom_column].apply(str)
        # Faz a conversão para um tipo apropriado ao GeoDataFrame
        df[geom_column] = df[geom_column].apply(shp.wkt.loads)
        if isinstance(crs, dict) and 'init' in crs:
            gdf = gpd.GeoDataFrame(df, geometry=geom_column, crs=crs)
        else:
            raise ValueError("É necessário informar o código EPSG no formato {'init': 'epsg:<int>'}")
        return gdf
    except:
        raise

def _insert_internal_title(column, ax):
    column_to_title = column.upper().replace('_', ' ')
    at = AnchoredText(column_to_title,
                      prop=dict(size=13), frameon=True,
                      loc='upper left',
                      )
    at.patch.set_boxstyle("square,pad=0.1")
    ax.add_artist(at)

def _insert_scale(artist_color, ax):
    ax.add_artist(ScaleBar(1, color=artist_color, height_fraction=0.004, 
        location=4, pad=1, frameon=False))

def _insert_north(artist_color, ax):
    x, y, arrow_length = 0.98, 0.985, 0.03
    ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
                arrowprops=dict(facecolor=artist_color, width=3, headwidth=8),
                ha='center', va='center', fontsize=10, color=artist_color,
                xycoords=ax.transAxes)

def make_smap(gdf, place, modeling, source, measures, crs):
    """
    Cria um dicionário que contém o GeoDataFrame e dados de propriedade intelectual
    Se um dataframe com uma coluna de geometria for passado, a função tentará converter
    para um GeoDataFrame
    
    Parâmetros
    ----------
    gdf : geopandas.GeoDataFrame
        Dicionário com GeoDataFrame e créditos : obrigatório
    place : str
        Nome do local (município, região metropolitana etc.) : obrigatório
    modeling : str
        Autor da modelagem : obrigatório
    source : str
        Fonte da modelagem (OSM, Imagem de satélite etc.) : obrigatório
    measures : str
        Responsável pelo cálculo das medidas sintáticas : obrigatório
    crs : str
        Sistema de coordenadas : obrigatório
    
    Retorno
    -------
    dict
    """
    
    # Verifica se o dataframe passado não é uma instância de geopandas.GeoDataFrame
    if not isinstance(gdf, gpd.GeoDataFrame):
        columns = gdf.columns.tolist()
        # Verifica se existem as colunas geom ou geometry no dataframe
        if 'geom' in columns:
            gdf = _load_geometry_column(gdf, crs=crs)
        elif 'geometry' in columns:
            gdf = _load_geometry_column(gdf, geom_column='geometry', crs=crs)
        else:
            raise ValueError("É necessária uma coluna de geometria ('geom' ou 'geometry')")
    
    # Cria o dicionário
    smap = {
        "data": gdf,
        "place": place,
        "modeling": modeling,
        "modeling_source": source,
        "syntactic_measures": measures,
        "coordinates_system": crs
    }
    
    return smap

def make_atlas(smap, cmap=cmap_default, bg_black=False):
    """
    Salva os mapas de todas as medidas
    
    Parâmetros
    ----------
    smap : dict
        Retorno de make_smap() : obrigatório
    cmap : str
        Paleta de cores
        Ver: https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html
    bg_black : bool
        Se verdadeiro, fundo do mapa será preto
    Retorno
    -------
    None
    """
    
    gdf = smap['data']
    
    # Lista de colunas para ignorar
    ignore = ['id', 'Depthmap_Ref', 'Axial_Line_Ref', 'axial_line_ref', 'depthmap_ref']
    
    # Filtra as colunas
    columns = gdf.columns.tolist()
    mcolumns = [c for c in columns if is_numeric_dtype(gdf[c]) and c not in ignore]
    
    # Salva cada um
    for m in mcolumns:
        plot_syntax_map(smap, column=m, cmap=cmap, bg_black=bg_black, save=True, show=False)
        print("Salvo mapa para {}".format(m))
